In [2]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv()

In [ ]:
X=df.iloc[:,:-1]
y=df.iloc[:,-1]

### Benchmark

In [ ]:
print('El benchmark es:',y[y == 1].shape[0]/y.shape[0])

EL benchmark es: 0.6229658536585366


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_nuevo, y, test_size=0.30, random_state=42)

### Logistic REgrerssion

In [ ]:
# Importamos la clase que contiene el algoritmo de regresíon logística

from sklearn.linear_model import LogisticRegression

# Instanciamos el modelo y lo entrenamos

clf = LogisticRegression()

# Lo entrenamos

clf.fit(X_train,y_train)

LogisticRegression()

### Decision Tree

In [ ]:
from sklearn.metrics import accuracy_score 

# Predecimos

y_pred = clf.predict(X_test)

# Evaluamos

print('El score del modelo es:', accuracy_score(y_test, y_pred))

El score del modelo es: 0.7115691056910569


In [ ]:
from sklearn.model_selection import GridSearchCV

clf = DecisionTreeClassifier() 
model = GridSearchCV(cv=5, error_score= np.nan,
             estimator=DecisionTreeClassifier(random_state = 42),
             param_grid={'ccp_alpha': [0.1, 0.01, 0.001],
                         'criterion': ['gini', 'entropy'],
                         'max_depth': [5, 6, 7, 8, 9],
                         'max_features': ['auto', 'sqrt', 'log2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=True)

In [ ]:
model.fit(X_train, y_train)

Fitting 5 folds for each of 90 candidates, totalling 450 fits


c:\Users\Carlos\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\tree\_classes.py:298: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
c:\Users\Carlos\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\tree\_classes.py:298: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
c:\Users\Carlos\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\tree\_classes.py:298: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
c:\Users\Carlos\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\tree\_classes.py:298: FutureWarning: `max_features='auto'` has been deprecated in

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'ccp_alpha': [0.1, 0.01, 0.001],
                         'criterion': ['gini', 'entropy'],
                         'max_depth': [5, 6, 7, 8, 9],
                         'max_features': ['auto', 'sqrt', 'log2']},
             verbose=True)

In [ ]:
model.best_estimator_

DecisionTreeClassifier(ccp_alpha=0.001, max_depth=9, max_features='log2')

In [ ]:
print("Mejores hiperparámetros: "+str(model.best_params_))
print("Mejor Score: "+str(model.best_score_)+'\n')

Mejores hiperparámetros: {'ccp_alpha': 0.001, 'criterion': 'gini', 'max_depth': 9, 'max_features': 'log2'}
Mejor Score: 0.6556933797909408

